---
Author: Mustapha Bouhsen <br>
[LinkedIn](https://www.linkedin.com/in/mustapha-bouhsen/)<br>
[Git](https://github.com/mus514)<br>
Date: February 2, 2024<br>
---

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime


### Load files from Azure blob storage : Set the data location and type


In [0]:
# storage_account_name = "mymlprojects"
# storage_key = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-03-09T09:14:29Z&st=2024-02-03T01:14:29Z&spr=https&sig=v%2Bmvq02eWWEzGfaXqGJ%2F8BJiTJrD3PPGS4eL66SIsC8%3D"

# container_name = "prod"
# mount_point = "/mnt/prod"

# dbutils.fs.mount(
#   source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
#   mount_point = mount_point,
#   extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net":storage_key})

In [0]:
#-----------------------------------------
# Set the the raw and the prof folder paths
#-----------------------------------------
raw_folder_path = "/mnt/raw/"
prod_folder_path = "/mnt/prod/"

In [0]:
#-----------------------------------------
# display the raw and the prof folder paths
#-----------------------------------------
print([file.path for file in dbutils.fs.ls(raw_folder_path)])
print(dbutils.fs.ls(prod_folder_path))

In [0]:
col_names = ["date", "open", "high", "low", "close", "volume"]

In [0]:
data = spark.read.json('dbfs:/mnt/raw/AAPL.json').collect()

In [0]:
df = pd.DataFrame(data[0][1].asDict()).T.reset_index()
df.columns = col_names
df[df["date"] >= "2020-01-01"].reset_index(drop=True).head(10)
